In [ ]:
!pip install -q --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.7 MB/s eta 0:00:00


In [ ]:
!pip install text_hammer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.31 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.9.1 which is incompatible.


In [ ]:
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_nlp
import keras_core as keras
from keras.optimizers import Adam
from tqdm import tqdm
import text_hammer as th
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

Using TensorFlow backend


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Carga de datos

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/disaster_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/disaster_test.csv")
print('Training Set Shape = {}'.format(df_train.shape))
print('Test Set Shape = {}'.format(df_test.shape))

Training Set Shape = (7613, 5)
Test Set Shape = (3263, 4)


In [ ]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
df_train["length"] = df_train["text"].apply(lambda x : len(x))
df_test["length"] = df_test["text"].apply(lambda x : len(x))

print("Train Length Stat")
print(df_train["length"].describe())
print()

print("Test Length Stat")
print(df_test["length"].describe())

Train Length Stat
count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64

Test Length Stat
count    3263.000000
mean      102.108183
std        33.972158
min         5.000000
25%        78.000000
50%       109.000000
75%       134.000000
max       151.000000
Name: length, dtype: float64


#Limpieza de datos

In [ ]:

def clean_dataset(dataframe, column):
  tqdm.pandas()
  dataframe[column] = dataframe[column].progress_apply(lambda x:str(x).lower())
  #dataframe[column] = dataframe[column].progress_apply(lambda x: th.remove_special_chars(x))
  dataframe[column] = dataframe[column].progress_apply(lambda x: re.sub("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", "", x))
  dataframe[column] = dataframe[column].progress_apply(lambda x: ' '.join(x.split()))

  return dataframe

In [ ]:
df_train = clean_dataset(df_train, 'text')
df_train.head()

100%|██████████| 7613/7613 [00:00<00:00, 333600.47it/s]


,id,keyword,location,text,target,length
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1,69
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,38
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1,88


In [ ]:
df_test = clean_dataset(df_test, 'text')

100%|██████████| 3263/3263 [00:00<00:00, 282255.69it/s]


In [ ]:
df_test.head()

,id,keyword,location,text,length
0,0,NaN,NaN,just happened a terrible car crash,34
1,2,NaN,NaN,"heard about #earthquake is different cities, s...",64
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96
3,9,NaN,NaN,apocalypse lighting. #spokane #wildfires,40
4,11,NaN,NaN,typhoon soudelor kills 28 in china and taiwan,45


In [ ]:
stop = stopwords.words("english")
df_train['text'] = df_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_test['text'] = df_test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_train.head()

,id,keyword,location,text,target,length
0,1,NaN,NaN,deeds reason #earthquake may allah forgive us,1,69
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,38
2,5,NaN,NaN,residents asked 'shelter place' notified offic...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,got sent photo ruby #alaska smoke #wildfires p...,1,88


In [ ]:
df_test.head()

,id,keyword,location,text,length
0,0,NaN,NaN,happened terrible car crash,34
1,2,NaN,NaN,"heard #earthquake different cities, stay safe ...",64
2,3,NaN,NaN,"forest fire spot pond, geese fleeing across st...",96
3,9,NaN,NaN,apocalypse lighting. #spokane #wildfires,40
4,11,NaN,NaN,typhoon soudelor kills 28 china taiwan,45


In [ ]:
def word_lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df_train['text'] = df_train['text'].apply(lambda x : word_lemmatizer(x))
df_test['text'] = df_train['text'].apply(lambda x : word_lemmatizer(x))
df_train.head()

,id,keyword,location,text,target,length
0,1,NaN,NaN,deed reason #earthquake may allah forgive u,1,69
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,38
2,5,NaN,NaN,resident asked 'shelter place' notified office...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,got sent photo ruby #alaska smoke #wildfires p...,1,88


#Precarga de parametros



In [ ]:
BATCH_SIZE = 32
NUM_TRAINING_EXAMPLES = df_train.shape[0]
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE

EPOCHS = 2
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
from sklearn.model_selection import train_test_split

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SPLIT, random_state=42)

X_test = df_test["text"]

#Entrenamiento de modelos

#distilBERT

In [ ]:
# Load a DistilBERT model.
preset= "distil_bert_base_en_uncased"

#vocab = ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"] + df_train['text'].tolist()
tokenizer = keras_nlp.models.DistilBertTokenizer.from_preset(preset)

tokenizer(df_train['text'].tolist())

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor(sequence_length=160,
                                                                   name="preprocessor_4_tweets",
                                                                   tokenizer=tokenizer)
# backbone = keras_nlp.models.DistilBertBackbone(
#     vocabulary_size=30552,
#     num_layers=4,
#     num_heads=4,
#     hidden_dim=256,
#     intermediate_dim=512,
#     max_sequence_length=128,
# )
# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=2)

#df_train['text'].tolist()
classifier.summary()

Preprocessor: "preprocessor_4_tweets"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer_11 (DistilBertTokenizer)     │                                              30,522 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "distil_bert_classifier_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                  ┃ Output Shape                           ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)                     │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ token_ids (InputLayer)                        │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ distil_bert_backbone_5 (DistilBertBackbone)   │ (None, None, 768)                      │      66,362,880 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ tf.__operators__.getitem_3 (SlicingOpLambda)  │ (None, 768)                            │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ pooled_dense (Dense)                          │ (None, 768)                            │         590,592 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ classifier_dropout (Dropout)                  │ (None, 768)                            │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ logits (Dense)                                │ (None, 2)                              │           1,538 │
└───────────────────────────────────────────────┴────────────────────────────────────────┴─────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'binary_crossentropy',
    optimizer=Adam(1e-5),
    metrics= ["accuracy"]
)


# Fit
history = classifier.fit(x=X_train,
                         y=y_train,
                         batch_size=BATCH_SIZE,
                         epochs=EPOCHS,
                         validation_data=(X_val, y_val)
                        )

Epoch 1/2
191/191 [==============================] - 7305s 38s/step - loss: 0.5073 - accuracy: 0.7644 - val_loss: 0.4200 - val_accuracy: 0.8221
Epoch 2/2
191/191 [==============================] - 7209s 38s/step - loss: 0.3845 - accuracy: 0.8397 - val_loss: 0.4132 - val_accuracy: 0.8345


In [ ]:
Xt = df_test["text"].values[0:32]
predictions = classifier.predict(Xt)

# Las predicciones son en forma de logits, puedes convertirlas en probabilidades usando softmax
predicted_probabilities = tf.nn.softmax(predictions)

# Obtén las clases predichas
predicted_classes = np.argmax(predicted_probabilities, axis=1)

1/1 [==============================] - 12s 12s/step


In [ ]:
predicted_classes

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0])

#BERT

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

dropout = tf.keras.layers.Dropout(0.3 , name="dropout")(pooled_output)
output = tf.keras.layers.Dense(2, activation='softmax' , name='output')(dropout)

model = tf.keras.Model(inputs=[text_input] , outputs=[output])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_word_ids': (None,    0         ['input_3[0][0]']             
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'binary_crossentropy',
    optimizer=Adam(1e-5),
    metrics= ["accuracy"])

history = model.fit(X_train, y_train , batch_size=BATCH_SIZE ,
                    epochs=EPOCHS , validation_data=(X_val, y_val))

Epoch 1/2
 14/191 [=>............................] - ETA: 1:32:44 - loss: 0.7563 - accuracy: 0.5402

# Small Bert

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/2",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

dropout = tf.keras.layers.Dropout(0.3 , name="dropout")(pooled_output)
output = tf.keras.layers.Dense(2 , activation='softmax' , name='output')(dropout)

model = tf.keras.Model(inputs=[text_input] , outputs=[output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['input_2[0][0]']             
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'binary_crossentropy',
    optimizer=Adam(1e-5),
    metrics= ["accuracy"])

history = model.fit(X_train, y_train , batch_size=BATCH_SIZE ,
                    epochs=4 , validation_data=(X_val, y_val))

Epoch 1/4
191/191 [==============================] - 1151s 6s/step - loss: 0.3298 - accuracy: 0.8631 - val_loss: 0.4564 - val_accuracy: 0.8129
Epoch 2/4
191/191 [==============================] - 1150s 6s/step - loss: 0.3086 - accuracy: 0.8729 - val_loss: 0.4651 - val_accuracy: 0.8129
Epoch 3/4
191/191 [==============================] - 1144s 6s/step - loss: 0.2747 - accuracy: 0.8898 - val_loss: 0.5227 - val_accuracy: 0.7971
Epoch 4/4
191/191 [==============================] - 1143s 6s/step - loss: 0.2486 - accuracy: 0.8998 - val_loss: 0.5556 - val_accuracy: 0.8004


#Albert

In [ ]:
print("somethinghhjgdhgdfdgyhjvfdgdfguioxcvhgfjhgjhgadsasdasds")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/albert/frameworks/TensorFlow2/variations/en-preprocess/versions/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/albert/frameworks/TensorFlow2/variations/en-base/versions/3",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

dropout = tf.keras.layers.Dropout(0.3 , name="dropout")(pooled_output)
output = tf.keras.layers.Dense(2 , activation='softmax' , name='output')(dropout)

model = tf.keras.Model(inputs=[text_input] , outputs=[output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['input_1[0][0]']             
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'binary_crossentropy',
    optimizer=Adam(1e-5),
    metrics= ["accuracy"])

history = model.fit(X_train, y_train , batch_size=BATCH_SIZE ,
                    epochs=4 , validation_data=(X_val, y_val))

Epoch 1/4


/tmp/__autograph_generated_filee8nlvtdl.py:58: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  ag__.converted_call(ag__.ld(warnings).warn, (f'"`{ag__.ld(fn_name)}` received `from_logits=True`, but the `output` argument was produced by a {ag__.ld(op_type)} activation and thus does not represent logits. Was this intended?',), dict(stacklevel=2), fscope)


191/191 [==============================] - 12079s 63s/step - loss: 0.5581 - accuracy: 0.7399 - val_loss: 0.4464 - val_accuracy: 0.8083
Epoch 2/4
191/191 [==============================] - 12006s 63s/step - loss: 0.4344 - accuracy: 0.8166 - val_loss: 0.5131 - val_accuracy: 0.7997
Epoch 3/4
191/191 [==============================] - ETA: 0s - loss: 0.3734 - accuracy: 0.8502 

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train['text'].values,
    df_train['target'].values,
    test_size=0.2,
    random_state=42
)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

train_padded = pad_sequences(train_sequences, maxlen=50, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=50, padding='post', truncating='post')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=16, input_length=50))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_padded,
    train_labels,
    validation_data=(val_padded, val_labels),
    epochs=4,
    batch_size=BATCH_SIZE
)


Epoch 1/4
191/191 [==============================] - 24s 88ms/step - loss: 0.6527 - accuracy: 0.6140 - val_loss: 0.5344 - val_accuracy: 0.7557
Epoch 2/4
191/191 [==============================] - 18s 93ms/step - loss: 0.4202 - accuracy: 0.8268 - val_loss: 0.4595 - val_accuracy: 0.7945
Epoch 3/4
191/191 [==============================] - 13s 69ms/step - loss: 0.2852 - accuracy: 0.8959 - val_loss: 0.5129 - val_accuracy: 0.7859
Epoch 4/4
191/191 [==============================] - 14s 74ms/step - loss: 0.2186 - accuracy: 0.9274 - val_loss: 0.5477 - val_accuracy: 0.7722
